In [49]:
import numpy as np
import pandas as pd
import numpy as np

## Loading the data

wet1 = pd.read_csv('C:/Users/ukesh/Desktop/CV/Internship/weather_train.csv')
wet1.head()

## First Step will be cleansing our data as we drop the values that have NA inside of just categorical variable so that it would be easier for compuation

weath=wet1.dropna(subset=['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow'])
weath.head()
weath.shape

## Which reduces our 130,891 rows to 111,374 rows

## Now we fill the NA with mean values for numerical values 

m1=weath.mean()
print(m1)

rain=weath.fillna(m1)
rain.head()

## Now just by having a look at the data we can know the redundant column that will be very easier if we filter out such as Sunshine, Date, Location Cloud 9am, Cloud 3pm and Evaporation


rain = rain.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','Date'],axis=1)
rain.shape


## Finally we have to change the categorical varibale Yes and No to 1 and 0 alongside the Wind Direction too using pandas get dummies

rain['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
rain['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)


## Changing the wind direction to numerical variables
categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am']
for col in categorical_columns:
    print(np.unique(rain[col]))
# transform the categorical columns
rain = pd.get_dummies(rain, columns=categorical_columns)
rain.iloc[4:9]


#next step is to standardize our data - using MinMaxScaler
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(rain)
rain = pd.DataFrame(scaler.transform(rain), index=rain.index, columns=rain.columns)
rain.iloc[4:10]
rain.shape
rain.head()

Unnamed: 0       74441.522213
MinTemp             12.420502
MaxTemp             23.477645
Rainfall             2.384288
Evaporation          5.604813
Sunshine             7.714198
WindGustSpeed       40.660208
WindSpeed9am        15.009392
WindSpeed3pm        19.164823
Humidity9am         67.676818
Humidity3pm         50.870079
Pressure9am       1017.406742
Pressure3pm       1015.019544
Cloud9am             4.407630
Cloud3pm             4.492314
Temp9am             17.275116
Temp3pm             21.900202
dtype: float64
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


,Unnamed: 0,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW
6,92976,8.0,20.4,0.2,30.0,11.0,15.0,47.0,47.0,1017.2,...,0,0,0,0,0,0,0,0,1,0
7,4150,15.0,27.1,0.2,24.0,7.0,15.0,89.0,57.0,1022.1,...,0,0,0,0,0,0,0,0,0,1
8,130392,12.9,21.5,0.0,56.0,17.0,33.0,60.0,32.0,1013.8,...,1,0,0,0,0,0,0,0,0,0
10,19119,14.6,21.0,1.4,39.0,7.0,13.0,79.0,64.0,1019.9,...,0,0,0,0,0,0,1,0,0,0
11,141371,26.6,34.5,0.6,56.0,9.0,24.0,60.0,57.0,1008.7,...,0,0,0,0,0,0,0,0,1,0


In [58]:
## STEP 2 FEATURE SELECTION

## So now we have to find out which feature impacts the most in terms of prediction
## We use the K best method from Scikit learn

from sklearn.feature_selection import SelectKBest, chi2

x = rain.loc[:,rain.columns!='RainTomorrow']
y = rain[['RainTomorrow']]
selector = SelectKBest(chi2, k=3)
selector.fit(x, y)
X_new = selector.transform(x)
print(x.columns[selector.get_support(indices=True)]) #top 3 columns

Index(['Rainfall', 'Humidity3pm', 'RainToday'], dtype='object')


In [66]:
## STEP 3 USING LOGISTIC REGRESSION AS PREDICTION OF WEATHER
## From Step 2 we can see that the most impactful variables are Rainfall, Humidity3pm and Rain Today

##So now we divide our variables consisiting of only impactful variables and negate the ones we don't require 


rain = rain[['Humidity3pm','Rainfall','RainToday','RainTomorrow']]
x = rain[['Humidity3pm']] 
y = rain[['RainTomorrow']]


#Logistic Regression 
## Import Scikit learn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Now we split the values to 70% in training and 30% into Test data
trainX,testX,trainY,testY = train_test_split(x,y,test_size=0.3)
## Apply Logistic regression
clf_Regression = LogisticRegression(random_state=0)
## Fit the data
clf_Regression.fit(trainX,trainY)
## Predict the value
prediction = clf_Regression.predict(testX)
score = accuracy_score(testY,prediction)
## Finally compute the accuracy of our model
print('Our Projects Test Accuracy is :',score*100)


print(prediction)



## From our answer we get the accuracy of 82.27% which is very good prediction accuracy.


Our Projects Test Accuracy is : 82.27935234788855
[0. 0. 0. ... 1. 0. 0.]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [67]:
print(prediction)

[0. 0. 0. ... 1. 0. 0.]


[[1.25179074 0.57772176 0.19119929 ... 0.25187163 0.23857909 0.23404682]
 [0.02436261 0.12882792 0.40558485 ... 0.25187163 0.23857909 0.23404682]
 [0.1628327  0.88248091 0.96840654 ... 0.25187163 0.23857909 0.23404682]
 ...
 [0.09806443 1.14939967 1.34313147 ... 0.25187163 0.23857909 0.23404682]
 [1.32289895 1.03949194 0.88513434 ... 0.25187163 0.23857909 0.23404682]
 [0.45646431 0.4585511  0.44722095 ... 0.25187163 0.23857909 0.23404682]]
(0, 63)


,Unnamed: 0,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW
